In [233]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from pandarallel import pandarallel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor


In [234]:

hp_s = pd.read_csv('price_paid_records.csv')
#Sample only 100,000 as data is large
#hp_s = hp.sample(n=100000)
# Get first 5 rows
hp_s.head()

,Transaction unique identifier,Price,Date of Transfer,Property Type,Old/New,Duration,Town/City,District,County,PPDCategory Type,Record Status - monthly file only
0,{81B82214-7FBC-4129-9F6B-4956B4A663AD},25000,1995-08-18 00:00,T,N,F,OLDHAM,OLDHAM,GREATER MANCHESTER,A,A
1,{8046EC72-1466-42D6-A753-4956BF7CD8A2},42500,1995-08-09 00:00,S,N,F,GRAYS,THURROCK,THURROCK,A,A
2,{278D581A-5BF3-4FCE-AF62-4956D87691E6},45000,1995-06-30 00:00,T,N,F,HIGHBRIDGE,SEDGEMOOR,SOMERSET,A,A
3,{1D861C06-A416-4865-973C-4956DB12CD12},43150,1995-11-24 00:00,T,N,F,BEDFORD,NORTH BEDFORDSHIRE,BEDFORDSHIRE,A,A
4,{DD8645FD-A815-43A6-A7BA-4956E58F1874},18899,1995-06-23 00:00,S,N,F,WAKEFIELD,LEEDS,WEST YORKSHIRE,A,A


In [235]:
#Checks if there are any null values
hp_s.isnull().sum()

Transaction unique identifier        0
Price                                0
Date of Transfer                     0
Property Type                        0
Old/New                              0
Duration                             0
Town/City                            0
District                             0
County                               0
PPDCategory Type                     0
Record Status - monthly file only    0
dtype: int64

In [236]:
#Drop columns that I will not need
hp_s.drop('Transaction unique identifier', axis = 1, inplace=True)
hp_s.drop('PPDCategory Type', axis = 1, inplace=True)
hp_s.drop('Record Status - monthly file only', axis = 1, inplace=True)
hp_s.drop('Duration', axis = 1, inplace=True)


In [237]:
hp_s.head()

,Price,Date of Transfer,Property Type,Old/New,Town/City,District,County
0,25000,1995-08-18 00:00,T,N,OLDHAM,OLDHAM,GREATER MANCHESTER
1,42500,1995-08-09 00:00,S,N,GRAYS,THURROCK,THURROCK
2,45000,1995-06-30 00:00,T,N,HIGHBRIDGE,SEDGEMOOR,SOMERSET
3,43150,1995-11-24 00:00,T,N,BEDFORD,NORTH BEDFORDSHIRE,BEDFORDSHIRE
4,18899,1995-06-23 00:00,S,N,WAKEFIELD,LEEDS,WEST YORKSHIRE


In [238]:
pandarallel.initialize(nb_workers=4)
hp_s['Date of Transfer'] = hp_s['Date of Transfer'].parallel_apply(lambda x : x.split(' ')[0]) 

hp_s['Year'] = hp_s['Date of Transfer'].str[:4]

hp_s['Date of Transfer'].iloc[1].split(" ")[0]

hp_s['Date of Transfer'] = hp_s['Date of Transfer'].parallel_apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))

hp_s['Year'] = hp_s['Date of Transfer'].dt.year
hp_s['Month'] = hp_s['Date of Transfer'].dt.month
hp_s.head()

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


,Price,Date of Transfer,Property Type,Old/New,Town/City,District,County,Year,Month
0,25000,1995-08-18,T,N,OLDHAM,OLDHAM,GREATER MANCHESTER,1995,8
1,42500,1995-08-09,S,N,GRAYS,THURROCK,THURROCK,1995,8
2,45000,1995-06-30,T,N,HIGHBRIDGE,SEDGEMOOR,SOMERSET,1995,6
3,43150,1995-11-24,T,N,BEDFORD,NORTH BEDFORDSHIRE,BEDFORDSHIRE,1995,11
4,18899,1995-06-23,S,N,WAKEFIELD,LEEDS,WEST YORKSHIRE,1995,6


In [239]:
print(f"Number of cities: {hp_s['Town/City'].nunique()}")
print(f"Number of transactions in Birmingham: {hp_s[hp_s['Town/City'] == 'BIRMINGHAM']['Price'].count()}")
print(f"Number of transactions in London: {hp_s[hp_s['Town/City'] == 'LONDON']['Price'].count()}")
print(f"Number of transactions in Wolverhampton: {hp_s[hp_s['Town/City'] == 'WOLVERHAMPTON']['Price'].count()}")


Number of cities: 1170
Number of transactions in Birmingham: 330358
Number of transactions in London: 1784194
Number of transactions in Wolverhampton: 77991


In [ ]:
#Top 50 results for most valued towns done based upon median values
hp_s.groupby(hp['Town/City'])['Price'].median().sort_values(ascending = False)[:50].plot(
    kind = "bar",grid = True, figsize = (16,6));

In [ ]:
#Most valued towns where most of the money is flowing into new builds
hp_new = hp_s[hp_s['Old/New'] == 'Y']

hp_new.groupby(hp['Town/City'])['Price'].sum().sort_values(ascending = False)[:50].plot(
    kind = "bar",grid = True, figsize = (16,6));

In [ ]:
#Drop Date of Transfer column
hp_s.drop(columns= 'Date of Transfer' , axis=1, inplace=True)

hp_s.head()

In [240]:
property_types = ("T", "S", "D", "F")
type_df = pd.DataFrame(hp_s, columns = ['Property Type'])

dum_df = pd.get_dummies(hp_s, columns = ["Property Type"], prefix = ["Property_Type_is_"] )

hp_s = type_df.join(dum_df)

hp_s.drop(columns = 'Property Type', axis = 1, inplace = True)


In [241]:
hp_s['Old/New'] = hp_s['Old/New'].map(lambda x:1 if x == 'Y' else 0)

In [242]:
#Can see the data is right skewed here
hp_s['Price'].skew()

91.49926338777554

In [243]:
log_price = np.log(hp_s['Price'])
hp_s['Price'] = log_price
print(log_price.skew())

-0.01846824565291894


In [244]:
pred_ds = hp_s.copy()

pred_ds.head()

,Price,Date of Transfer,Old/New,Town/City,District,County,Year,Month,Property_Type_is__D,Property_Type_is__F,Property_Type_is__O,Property_Type_is__S,Property_Type_is__T
0,10.126631,1995-08-18,0,OLDHAM,OLDHAM,GREATER MANCHESTER,1995,8,0,0,0,0,1
1,10.657259,1995-08-09,0,GRAYS,THURROCK,THURROCK,1995,8,0,0,0,1,0
2,10.714418,1995-06-30,0,HIGHBRIDGE,SEDGEMOOR,SOMERSET,1995,6,0,0,0,0,1
3,10.672438,1995-11-24,0,BEDFORD,NORTH BEDFORDSHIRE,BEDFORDSHIRE,1995,11,0,0,0,0,1
4,9.846864,1995-06-23,0,WAKEFIELD,LEEDS,WEST YORKSHIRE,1995,6,0,0,0,1,0


In [245]:
pred_ds['Town/City'] = pred_ds['Town/City'].factorize()[0].astype('float32')
pred_ds['District'] = pred_ds['District'].factorize()[0].astype('float32')
pred_ds['County'] = pred_ds['County'].factorize()[0].astype('float32')


In [246]:
pred_ds

,Price,Date of Transfer,Old/New,Town/City,District,County,Year,Month,Property_Type_is__D,Property_Type_is__F,Property_Type_is__O,Property_Type_is__S,Property_Type_is__T
0,10.126631,1995-08-18,0,0.0,0.0,0.0,1995,8,0,0,0,0,1
1,10.657259,1995-08-09,0,1.0,1.0,1.0,1995,8,0,0,0,1,0
2,10.714418,1995-06-30,0,2.0,2.0,2.0,1995,6,0,0,0,0,1
3,10.672438,1995-11-24,0,3.0,3.0,3.0,1995,11,0,0,0,0,1
4,9.846864,1995-06-23,0,4.0,4.0,4.0,1995,6,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22489343,12.072541,2017-02-20,0,20.0,4.0,4.0,2017,2,0,0,0,1,0
22489344,13.282686,2017-02-15,0,213.0,4.0,4.0,2017,2,1,0,0,0,0
22489345,12.520883,2017-02-24,0,150.0,144.0,4.0,2017,2,1,0,0,0,0
22489346,10.491274,2017-02-22,0,165.0,32.0,4.0,2017,2,0,0,0,0,1


In [247]:
pred_ds.drop(columns='Date of Transfer', axis=1, inplace=True)

In [248]:
pred_ds.head()

,Price,Old/New,Town/City,District,County,Year,Month,Property_Type_is__D,Property_Type_is__F,Property_Type_is__O,Property_Type_is__S,Property_Type_is__T
0,10.126631,0,0.0,0.0,0.0,1995,8,0,0,0,0,1
1,10.657259,0,1.0,1.0,1.0,1995,8,0,0,0,1,0
2,10.714418,0,2.0,2.0,2.0,1995,6,0,0,0,0,1
3,10.672438,0,3.0,3.0,3.0,1995,11,0,0,0,0,1
4,9.846864,0,4.0,4.0,4.0,1995,6,0,0,0,1,0


In [249]:
sep_cols = ['District' , 'County', 'Old/New', 'Town/City','Year', 'Month','Property_Type_is__D', 'Property_Type_is__F',
            'Property_Type_is__O', 'Property_Type_is__S', 'Property_Type_is__T']

In [250]:
x = pred_ds[sep_cols]
y = pred_ds['Price']

In [251]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 0)


In [252]:
scaler = StandardScaler()

normalized_array = scaler.fit_transform(x_train[sep_cols])
normalized_x_train = pd.DataFrame(normalized_array, columns=sep_cols)

normalized_x_train

,District,County,Old/New,Town/City,Year,Month,Property_Type_is__D,Property_Type_is__F,Property_Type_is__O,Property_Type_is__S,Property_Type_is__T
0,-1.315166,-0.853415,2.966116,-0.959355,1.245737,-1.426942,-0.546458,2.123151,-0.066999,-0.618031,-0.666568
1,1.763185,2.340666,-0.337141,-0.769854,1.245737,0.371849,-0.546458,2.123151,-0.066999,-0.618031,-0.666568
2,-0.190238,-0.853415,-0.337141,-0.959355,-1.290728,0.671648,-0.546458,2.123151,-0.066999,-0.618031,-0.666568
3,0.820677,0.160091,-0.337141,0.166281,-0.656611,0.971446,1.829968,-0.470998,-0.066999,-0.618031,-0.666568
4,-0.053422,-0.945552,-0.337141,0.992506,-0.181024,1.571044,-0.546458,-0.470998,-0.066999,-0.618031,1.500223
...,...,...,...,...,...,...,...,...,...,...,...
16867006,1.747983,2.279241,-0.337141,-0.451492,0.453092,0.072051,-0.546458,-0.470998,-0.066999,-0.618031,1.500223
16867007,-0.091427,-0.791990,-0.337141,0.480853,-0.815141,1.271245,1.829968,-0.470998,-0.066999,-0.618031,-0.666568
16867008,-0.767904,-0.730566,-0.337141,0.026051,-1.290728,0.371849,-0.546458,-0.470998,-0.066999,-0.618031,1.500223
16867009,1.542760,1.818556,-0.337141,0.821955,1.087208,0.072051,-0.546458,2.123151,-0.066999,-0.618031,-0.666568


In [253]:
print(x_train.shape, y_train.shape)


(16867011, 11) (16867011,)


In [254]:
kf = KFold(n_splits = 5, random_state = None)

lreg = LinearRegression()
     
avg_lreg_score = cross_val_score(lreg , normalized_x_train, y_train, cv = kf)


In [255]:
print('Avg accuracy : {}'.format(avg_lreg_score))


Avg accuracy : [0.40331456 0.40339153 0.40251539 0.40338231 0.4031845 ]


In [256]:
kf = KFold(n_splits = 5, random_state = None)

tr_regressor = DecisionTreeRegressor(random_state = 0)

avg_tr_score = cross_val_score(tr_regressor, normalized_x_train, y_train, cv = kf)

In [257]:
print('Avg accuracy : {}'.format(avg_tr_score))


Avg accuracy : [0.72356614 0.72333277 0.72213338 0.72342218 0.72423134]
